In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
from sklearn.metrics import mean_squared_error
from lcdb.db import LCDB
from lcdb.analysis.util import LearningCurveExtractor, merge_curves

import sys
sys.path.append("../../2022-ecml/analysis")
# from utils_fit import get_anchors_and_scores_mean_curve, convert_table2, remove_rows_with_nan_or_inf, load_from_parts, remove_bad_fits, failed_fits_statistics,  prepare_total_dataframe, plot_data2, plot_trn_data2, plot_prediction2, plot_prediction_smooth2, get_fun_model_id, get_XY2, get_ranks_from_tables, build_rank_table, convert_to_cd_tables, filter_table, prepare_data_for_cd, make_all_cd_plots, print_pretty_rank_table_transpose, plot_metric, get_relevant_max_anchor, empirical_cdf
from fit_database import fit_model#, get_multiple_extrapolations_mean_curve_robust, df_compute_metrics_mean_curve

In [4]:
workflows = [
    "lcdb.workflow.sklearn.KNNWorkflow",
    "lcdb.workflow.sklearn.LibLinearWorkflow",
    "lcdb.workflow.sklearn.LibSVMWorkflow",
    "lcdb.workflow.sklearn.TreesEnsembleWorkflow"
]

openmlid = 6    # 3,6,11,12,14
workflow = workflows[1]

# retrieve learning curve objects
lcdb = LCDB()
df = lcdb.query(
    openmlids=[openmlid],
    workflows=[workflow],
    return_generator=False,
    processors={
        "learning_curve": LearningCurveExtractor(
            metrics=["error_rate"],
            folds=["train", "val"]
        )
    },
    show_progress=True
)

# group by configurations
config_cols = [c for c in df.columns if c.startswith("p:")]
df = df.groupby(config_cols).agg({"learning_curve": merge_curves})

100%|██████████| 25/25 [04:40<00:00, 11.23s/it]


In [5]:
max_schedule = df["learning_curve"].iloc[0].anchors_size
print(max_schedule)

[16, 23, 32, 45, 64, 91, 128, 181, 256, 362, 512, 724, 1024, 1448, 2048, 2896, 4096, 5793, 8192, 11585, 16000]


In [6]:
df["learning_curve"].apply(lambda x: x.pad_anchors_size(max_schedule, inplace=True))

p:C           p:class_weight  p:dual  p:fit_intercept  p:intercept_scaling  p:loss         p:max_iter  p:multiclass  p:penalty  p:pp@cat_encoder  p:pp@decomposition  p:pp@featuregen  p:pp@featureselector  p:pp@scaler  p:tol     p:pp@kernel_pca_kernel  p:pp@kernel_pca_n_components  p:pp@poly_degree  p:pp@selectp_percentile  p:pp@std_with_std
1.100000e-12  balanced        False   False            468.603195           squared_hinge  2078        ovo-scikit    l1         ordinal           agglomerator        none             selectp               minmax       0.007298  linear                  0.250000                      2                 39                       True                 <lcdb.analysis.util.LearningCurve object at 0x...
                              True    False            1.310196             squared_hinge  149         ovr           l2         ordinal           lda                 none             selectp               std          0.000356  linear                  0.250000 

## Parametric model fitting

In [7]:
model_group = ['pow4', 'mmf4', 'wbl4', 'exp4', 'last1']

In [12]:
def curves_models_fitting(lc_data, model_names, extrapolate, input_mask, rep=5, verbose=True):
    fitting_results = []

    # counters for successful fits
    model_fit_stats = {model_name: {"successful_fits": 0, "total_fits": 0} for model_name in model_names}

    pbar = tqdm(total=len(lc_data) * len(model_names), smoothing=0, miniters=1)
    for lc in lc_data:
        schedule = lc.anchors_size  
        
        # ensure no NaN values in lc
        if np.any(~np.isnan(lc.values)):
            # compute mean
            lc = lc.values[0, 1, :, :, :, :, -1] if lc.is_iteration_wise_curve else lc.values[0, 1]
            scores = np.nanmean(lc, axis=(0, 1, 2)).squeeze() 
            
            for model_name in model_names:
                model_fit_stats[model_name]["total_fits"] += 1  

                try:    # fitting
                    if extrapolate: 
                        train_schedule = np.array(schedule)[:-input_mask] # training length
                        regress_target = np.array(scores)[:-input_mask]
                    else: 
                        train_schedule = np.array(schedule) 
                        regress_target = np.array(scores)
                    beta, model, fails_init, fails_fit = fit_model(
                        train_schedule, regress_target, 
                        np.array(schedule),  # extrapolation length
                        model_name, rep=rep, verbose=verbose)
                    
                    # predictions 
                    predictions = model(np.array(schedule))
                    #########################
                    predictions[predictions>1] = 1
                    predictions[predictions<0] = 0
                    #########################

                    #MSE between predictions and actual scores
                    if extrapolate: 
                        mse = mean_squared_error(np.array(scores)[-input_mask:], predictions[-input_mask:])
                    else: # whole curve
                        mse = mean_squared_error(np.array(scores), predictions)

                    fitting_results.append({
                        "schedule": schedule,
                        "scores": scores,
                        "predictions": predictions,
                        "mse": mse,      
                        "curve_model": model_name,
                        "beta": beta,
                        "fails_init": fails_init,
                        "fails_fit": fails_fit
                    })

                    model_fit_stats[model_name]["successful_fits"] += 1

                except Exception as e:
                    # failed fitting
                    if verbose:
                        print(f"Failed to fit model {model_name} for current learning curve. Error: {e}")
                
                pbar.update(1)

        else:
            # skip NaN values learning curve
            continue

    pbar.close()
    return fitting_results, model_fit_stats

In [13]:
fitting_results, fit_stats = curves_models_fitting(df["learning_curve"], model_group, extrapolate=False, input_mask=0)
result_df = pd.DataFrame(fitting_results)

print("Results:")
for model_name in model_group:
    model_data = result_df[result_df['curve_model'] == model_name]
    if len(model_data) > 0:
        avg_mse = model_data["mse"].mean()
        success_rate = (fit_stats[model_name]["successful_fits"] / fit_stats[model_name]["total_fits"]) * 100
        print(f"Model: {model_name}")
        print(f"  Average MSE: {avg_mse}")
        print(f"  Fitting rate: {success_rate:.2f}% with {fit_stats[model_name]['successful_fits']} successful fits in total {fit_stats[model_name]['total_fits']}")


100%|██████████| 6155/6155 [02:59<00:00, 34.32it/s]

Results:
Model: pow4
  Average MSE: 0.0005885709280283777
  Fitting rate: 100.00% with 1231 successful fits in total 1231
Model: mmf4
  Average MSE: 0.00043792136506871394
  Fitting rate: 100.00% with 1231 successful fits in total 1231
Model: wbl4
  Average MSE: 0.002226175384670101
  Fitting rate: 100.00% with 1231 successful fits in total 1231
Model: exp4
  Average MSE: 0.004147834537795348
  Fitting rate: 100.00% with 1231 successful fits in total 1231
Model: last1
  Average MSE: 0.0549946583121166
  Fitting rate: 100.00% with 1231 successful fits in total 1231


### Extrapolate 2 anchor i.e. 50%

In [14]:
fitting_results, fit_stats = curves_models_fitting(df["learning_curve"], model_group, extrapolate=True, input_mask=2)
result_df = pd.DataFrame(fitting_results)

print("Results:")
for model_name in model_group:
    model_data = result_df[result_df['curve_model'] == model_name]
    if len(model_data) > 0:
        avg_mse = model_data["mse"].mean()
        success_rate = (fit_stats[model_name]["successful_fits"] / fit_stats[model_name]["total_fits"]) * 100
        print(f"Model: {model_name}")
        print(f"  Average MSE: {avg_mse}")
        print(f"  Fitting rate: {success_rate:.2f}% with {fit_stats[model_name]['successful_fits']} successful fits in total {fit_stats[model_name]['total_fits']}")


  0%|          | 21/6155 [00:00<02:19, 44.12it/s]

fit failed, nan error? True inf error? False model? mmf4


  1%|          | 36/6155 [00:00<02:24, 42.39it/s]

fit failed, nan error? True inf error? False model? mmf4


  2%|▏         | 111/6155 [00:02<02:34, 39.20it/s]

...

fit failed, nan error? True inf error? False model? mmf4


100%|█████████▉| 6149/6155 [03:09<00:00, 32.44it/s]

fit failed, nan error? True inf error? False model? mmf4


100%|██████████| 6155/6155 [03:09<00:00, 32.45it/s]

Results:
Model: pow4
  Average MSE: 0.002370996065291243
  Fitting rate: 100.00% with 1231 successful fits in total 1231
Model: mmf4
  Average MSE: 0.002368372401813456
  Fitting rate: 100.00% with 1231 successful fits in total 1231
Model: wbl4
  Average MSE: 0.006040579360941095
  Fitting rate: 100.00% with 1231 successful fits in total 1231
Model: exp4
  Average MSE: 0.010133039525823787
  Fitting rate: 100.00% with 1231 successful fits in total 1231
Model: last1
  Average MSE: 0.0015124974263626618
  Fitting rate: 100.00% with 1231 successful fits in total 1231


### Extrapolate 4 anchor i.e. 25%

In [15]:
fitting_results, fit_stats = curves_models_fitting(df["learning_curve"], model_group, extrapolate=True, input_mask=4)
result_df = pd.DataFrame(fitting_results)

print("Results:")
for model_name in model_group:
    model_data = result_df[result_df['curve_model'] == model_name]
    if len(model_data) > 0:
        avg_mse = model_data["mse"].mean()
        success_rate = (fit_stats[model_name]["successful_fits"] / fit_stats[model_name]["total_fits"]) * 100
        print(f"Model: {model_name}")
        print(f"  Average MSE: {avg_mse}")
        print(f"  Fitting rate: {success_rate:.2f}% with {fit_stats[model_name]['successful_fits']} successful fits in total {fit_stats[model_name]['total_fits']}")


  2%|▏         | 111/6155 [00:04<03:54, 25.79it/s]

fit failed, nan error? True inf error? False model? mmf4


  5%|▍         | 307/6155 [00:09<02:56, 33.15it/s]

fit failed, nan error? True inf error? False model? mmf4


  6%|▋         | 391/6155 [00:11<02:53, 33.16it/s]

...

fit failed, nan error? True inf error? False model? mmf4


100%|█████████▉| 6138/6155 [03:12<00:00, 31.91it/s]

fit failed, nan error? True inf error? False model? mmf4


100%|██████████| 6155/6155 [03:13<00:00, 31.85it/s]

Results:
Model: pow4
  Average MSE: 0.0074859046232745785
  Fitting rate: 100.00% with 1231 successful fits in total 1231
Model: mmf4
  Average MSE: 0.005327899605015675
  Fitting rate: 100.00% with 1231 successful fits in total 1231
Model: wbl4
  Average MSE: 0.008742958579354638
  Fitting rate: 100.00% with 1231 successful fits in total 1231
Model: exp4
  Average MSE: 0.012985658756851223
  Fitting rate: 100.00% with 1231 successful fits in total 1231
Model: last1
  Average MSE: 0.0026743451329384674
  Fitting rate: 100.00% with 1231 successful fits in total 1231
